# Assignment 3 group 25

By:
Alexandra Bock, Elena Ecker, Alessandro Maugeri, Giorgio Rinaldi

In [1]:
import math
import sys
import pandas as pd
import numpy as np
from datetime import date, datetime
import string
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk import punkt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Extraction and Manipulation

Here are represented the historical data of Tesla stock

In [2]:
tesla = pd.read_csv('TSLA Historical Data.csv')[::-1]
print(tesla.head())
print(tesla.info())

              Date Price  Open  High   Low   Vol. Change %
2484  Jan 03, 2012  5.62  5.79  5.90  5.53  4.64M   -1.58%
2483  Jan 04, 2012  5.54  5.64  5.73  5.50  3.15M   -1.42%
2482  Jan 05, 2012  5.42  5.55  5.59  5.37  5.03M   -2.17%
2481  Jan 06, 2012  5.38  5.44  5.56  5.28  4.93M   -0.74%
2480  Jan 09, 2012  5.45  5.40  5.50  5.22  4.48M    1.30%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 2484 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      2485 non-null   object
 1   Price     2485 non-null   object
 2   Open      2485 non-null   object
 3   High      2485 non-null   object
 4   Low       2485 non-null   object
 5   Vol.      2485 non-null   object
 6   Change %  2485 non-null   object
dtypes: object(7)
memory usage: 136.0+ KB
None


We are getting rid of useless information and resorting the dataframe by date

In [3]:
tesla['date']= pd.to_datetime(tesla['Date']).dt.date
tesla = tesla.drop(axis=1, columns=['Price', 'Open', 'High', 'Low', 'Vol.', 'Date'])
tesla = tesla.sort_values(by='date').reset_index(drop=True)
print(tesla.head())

  Change %        date
0   -1.58%  2012-01-03
1   -1.42%  2012-01-04
2   -2.17%  2012-01-05
3   -0.74%  2012-01-06
4    1.30%  2012-01-09


Elon tweets since 2012

In [4]:
data_url = 'https://raw.githubusercontent.com/NuttySalmon/Elon-Musk-Tweets-VS-Tesla-Stock/master/tweets.csv'
elon = pd.read_csv(data_url)
print(elon.head())

   username   to                                               text  retweets  \
0  elonmusk  NaN  Just test-fired the Superdraco rocket engine. ...        63   
1  elonmusk  NaN  Um, while awesome in its own way, that was not...         5   
2  elonmusk  NaN  The SpaceX theme song by Total Ghost is just t...        32   
3  elonmusk  NaN  The lady doth protest too little. http://finan...         4   
4  elonmusk  NaN  The Model X unveiling will be webcast live on ...       109   

   favorites  replies                  id  \
0         67       13  164740558374445056   
1         19       12  164637208769339392   
2         79       14  164389972688908289   
3          2        1  163349291790893056   
4         23       18  163084140386861056   

                                           permalink  author_id  \
0  https://twitter.com/elonmusk/status/1647405583...   44196397   
1  https://twitter.com/elonmusk/status/1646372087...   44196397   
2  https://twitter.com/elonmusk/status/16

Getting rid of useless information about tweets

First goal: transform the types of data in order to process them. First let's analyze the the info about this csv.

In [5]:
print(elon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9407 entries, 0 to 9406
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   username        9407 non-null   object 
 1   to              6717 non-null   object 
 2   text            9314 non-null   object 
 3   retweets        9407 non-null   int64  
 4   favorites       9407 non-null   int64  
 5   replies         9407 non-null   int64  
 6   id              9407 non-null   int64  
 7   permalink       9407 non-null   object 
 8   author_id       9407 non-null   int64  
 9   date            9407 non-null   object 
 10  formatted_date  9407 non-null   object 
 11  hashtags        42 non-null     object 
 12  mentions        808 non-null    object 
 13  geo             0 non-null      float64
 14  urls            1251 non-null   object 
dtypes: float64(1), int64(5), object(9)
memory usage: 1.1+ MB
None


Text will be treated as string, date as date (we are removing the time). adding day, month, year columns. resorting by date

In [6]:
elon = elon.drop(axis=1, columns=['retweets','favorites','replies','to', 'hashtags','author_id','permalink','mentions','geo','urls','id','username', 'formatted_date'])
elon['text'] = elon['text'].astype('string')
elon['date']= pd.to_datetime(elon['date']).dt.date
elon = elon.sort_values(by='date').reset_index(drop=True)
print(elon.head())

                                                text        date
0  Just finished Massie's book on Catherine. An a...  2012-01-03
1  What everyone really thinks ... RT “@jonlovett...  2012-01-03
2  @om Respectfully disagree. Larry made awesome ...  2012-01-03
3  Next month is also when our Dragon spaceship d...  2012-01-06
4      Model S Signature series sold out as of today  2012-01-06


Merge the two datasets linking by the same date

In [7]:
merge_df = pd.merge(tesla, elon,  how='left', left_on=['date'], right_on = ['date'])

The 'date' column now is no more usefull since we've the correspondence between the change in stock and the text of each tweet in the corresponding day. These two columns represent our X and Y, but first need to be manipulated.
In particular, we remove all the days in which there were no tweets by Musk (dropna).

In [8]:
elon = merge_df.drop(axis=1, columns=['date']).dropna(axis=0)
print(elon.head())

  Change %                                               text
0   -1.58%  Just finished Massie's book on Catherine. An a...
1   -1.58%  What everyone really thinks ... RT “@jonlovett...
2   -1.58%  @om Respectfully disagree. Larry made awesome ...
5   -0.74%  Next month is also when our Dragon spaceship d...
6   -0.74%      Model S Signature series sold out as of today


With lambda function applied to the 'Change %', positive trend will be classified as 1 and negative with -1

In [9]:
elon['Label'] = elon['Change %'].apply(lambda x: '1' if '-' not in x  else '-1')
print(elon.tail())

     Change %                                               text Label
7361  -10.31%                                            Exactly    -1
7362  -10.31%                                          Well said    -1
7363  -10.31%                                                       -1
7364  -10.31%    Oh, say does that star-spangled banner yet wave    -1
7365  -10.31%  Don’t need the cash. Devoting myself to Mars a...    -1


Even here, we apply lambda to tokenize all the rows of text.

In [10]:
elon['tokenized_sents'] = elon.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
print(elon.head())

  Change %                                               text Label  \
0   -1.58%  Just finished Massie's book on Catherine. An a...    -1   
1   -1.58%  What everyone really thinks ... RT “@jonlovett...    -1   
2   -1.58%  @om Respectfully disagree. Larry made awesome ...    -1   
5   -0.74%  Next month is also when our Dragon spaceship d...    -1   
6   -0.74%      Model S Signature series sold out as of today    -1   

                                     tokenized_sents  
0  [Just, finished, Massie, 's, book, on, Catheri...  
1  [What, everyone, really, thinks, ..., RT, “, @...  
2  [@, om, Respectfully, disagree, ., Larry, made...  
5  [Next, month, is, also, when, our, Dragon, spa...  
6  [Model, S, Signature, series, sold, out, as, o...  


Lemmatization required the creation of a function since we had to iterate over each element of a cell.


In [11]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
elon['lemmatize_sents'] = elon['tokenized_sents'].apply(lemmatize_text)
print(elon['lemmatize_sents'])

0       [Just, finished, Massie, 's, book, on, Catheri...
1       [What, everyone, really, think, ..., RT, “, @,...
2       [@, om, Respectfully, disagree, ., Larry, made...
5       [Next, month, is, also, when, our, Dragon, spa...
6       [Model, S, Signature, series, sold, out, a, of...
                              ...                        
7361                                            [Exactly]
7362                                         [Well, said]
7363                                                   []
7364    [Oh, ,, say, doe, that, star-spangled, banner,...
7365    [Don, ’, t, need, the, cash, ., Devoting, myse...
Name: lemmatize_sents, Length: 6395, dtype: object


We extract the positive and negative tweets and transform them to lists.
We apply a function to remove punctuation and stop words.

In [12]:
negative_post = elon[elon['Label'] == '-1']
negative_post = negative_post['lemmatize_sents'].copy()
negative_post = negative_post.to_list()



positive_post = elon[elon['Label'] == '1']
positive_post = positive_post['lemmatize_sents'].copy()
positive_post = positive_post.to_list()

punctuation_stop = ['\'s','.','(', ')', ',', ':', ';', '’','’s', '@', '#', '!', '?', '&','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

def remove_p_s(x):
    x1 = list()
    for y in x:
        x2 = []
        for z in y:
            if z not in punctuation_stop and len(z) > 1:
                x2.append(z.lower())
        x1.append(x2)
    return x1            
         
negative_post = remove_p_s(negative_post)
positive_post = remove_p_s(positive_post)


negative_post = [" ".join(x) for x in negative_post]
positive_post = [" ".join(x) for x in positive_post]
print(negative_post[:5], positive_post[:5])

['just finished massie book catherine an amazingly expansive compelling portrait incredible woman highly recommend', "what everyone really think ... rt jonlovett if mayans good predicting future 'd mayans", 'om respectfully disagree larry made awesome move last year laid foundation g+ kick butt 2012 he stud', 'next month also dragon spaceship dock space station first time major pucker factor ...', 'model signature series sold today'] ['the exec conf room tesla used called denali decided move letter around seemed apt', 'model performance powertrain produced much torque today broke dyno 4.4 sec 0-60 mph problem', "kids day grown rt `` second-graders wow audience with school production of equus ''", 'what magical cowboy herding cat livin dream ...', 'tsla 500m friday back 500m today sorry roller coaster']


## Modelling and Processing

We now apply the td-idf technique to extract information about words. We will get a matrix in which as rows we have the single tweets and as columns all the unique words in all tweets.
We then append our y, label +1 since it is positive

In [13]:

tfidf = TfidfVectorizer(min_df=2,max_df=0.5, ngram_range=(1,3))
features = tfidf.fit_transform(positive_post)
df_Positive = pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names()
)
df_Positive['Label'] = '1'
print(df_Positive)

      000  000 km  000 lb  000 people   02   04   07   09   10  10 year  ...  \
0     0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
1     0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
2     0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
3     0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
4     0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
...   ...     ...     ...         ...  ...  ...  ...  ...  ...      ...  ...   
3460  0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
3461  0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
3462  0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
3463  0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   
3464  0.0     0.0     0.0         0.0  0.0  0.0  0.0  0.0  0.0      0.0  ...   

      youtube com watch  yup   ze  ze e

Here the opposite, -1 since the values were negative durign the days tweets were posted.

In [14]:
tfidf = TfidfVectorizer(min_df=2,max_df=0.5, ngram_range=(1,3))
features = tfidf.fit_transform(negative_post)
df_Negative = pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names()
)
df_Negative['Label'] = '-1'
df_Negative.head()


print(df_Negative)

      000  000 km  000 lb  000 mile  000 people   01   03   05   07   10  ...  \
0     0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
1     0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
2     0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
3     0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
4     0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
...   ...     ...     ...       ...         ...  ...  ...  ...  ...  ...  ...   
2925  0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
2926  0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
2927  0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
2928  0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   
2929  0.0     0.0     0.0       0.0         0.0  0.0  0.0  0.0  0.0  0.0  ...   

      youtu  youtu be  yout

Now we get positive and negative reviews back together, and we transform types in np.float32 in order to process them in order to satisfy the needs of the algorithms we are going to use.

In [15]:
df = df_Positive.append(df_Negative)
df = df.fillna(0.0).astype(np.float32)



In [16]:
x = df.drop('Label',axis=1)
y = df['Label']

x.shape, y.shape

((6395, 6565), (6395,))

Training and test split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 0, stratify = y)

Random Forest Classifier

In [18]:
def runRandomForest(x_train, x_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    cr = classification_report(y_test, y_pred)
    print(cr)

SVC

In [19]:
def SVC(x_train, x_test, y_train, y_test): 
    clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    cr = classification_report(y_test, y_pred)
    print(cr)

LSVC

In [20]:
def LSVC(x_train, x_test, y_train, y_test):
    lsvc = LinearSVC(verbose=0)
    print(lsvc)

    LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
              intercept_scaling=1, loss='squared_hinge', max_iter=1000,
              multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
              verbose=0)


    lsvc.fit(x_train, y_train)
    y_pred = lsvc.predict(x_test)
    cr = classification_report(y_test, y_pred)
    print(cr)

## Results

In [21]:
runRandomForest(x_train, x_test, y_train, y_test)
LSVC(x_train, x_test, y_train, y_test)
SVC(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

        -1.0       0.51      0.33      0.40       586
         1.0       0.56      0.73      0.63       693

    accuracy                           0.55      1279
   macro avg       0.53      0.53      0.52      1279
weighted avg       0.54      0.55      0.53      1279

LinearSVC()
              precision    recall  f1-score   support

        -1.0       0.60      0.54      0.57       586
         1.0       0.64      0.70      0.67       693

    accuracy                           0.63      1279
   macro avg       0.62      0.62      0.62      1279
weighted avg       0.63      0.63      0.63      1279

              precision    recall  f1-score   support

        -1.0       0.58      0.44      0.50       586
         1.0       0.61      0.73      0.66       693

    accuracy                           0.60      1279
   macro avg       0.59      0.58      0.58      1279
weighted avg       0.59      0.60      0.59      1279

